In [1]:
from collections import deque, defaultdict

import numpy as np

In [2]:
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [3]:
def get_region_plant_start(grid, plant, start):

    assert grid[start] == plant

    n_rows, n_cols = grid.shape

    visited = set()
    n_connexions = 0
    queue = deque([start])

    while queue:
        x_current, y_current = queue.popleft()
        if (x_current, y_current) in visited:
            continue
        visited.add((x_current, y_current))

        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy
            if 0 <= x_next < n_rows and 0 <= y_next < n_cols and (x_next, y_next) not in visited:
                if grid[x_next, y_next] == plant:
                    queue.append((x_next, y_next))
                    n_connexions += 1

    return visited, n_connexions

In [4]:
def get_all_regions_one_plant(grid, plant):
    plant_pos = set([tuple(pos.tolist()) for pos in np.argwhere(grid == plant)])
    plant_regions = []

    while plant_pos:
        start = plant_pos.pop()
        region, n_connexions = get_region_plant_start(grid, plant, start)
        plant_regions.append((str(plant), n_connexions, region))
        plant_pos = plant_pos.difference(region)

    return plant_regions

In [5]:
def get_all_regions(grid):
    regions = []
    for plant in np.unique(grid):
        regions.extend(get_all_regions_one_plant(grid, plant))
    return regions

In [6]:
def compute_total_cost_p1(grid):
    total_cost = 0
    for plant, n_connexions, coords in get_all_regions(grid):
        n_plots = len(coords)
        area = n_plots
        perimeter = 4 * n_plots - 2 * n_connexions
        total_cost += area * perimeter
    return total_cost

In [7]:
def cound_sides_region(grid, plant, coords):
    n_rows, n_cols = grid.shape

    x_sides = set()
    y_sides = set()

    for x_current, y_current in coords:
        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy
            if 0 <= x_next < n_rows and 0 <= y_next < n_cols:
                if grid[x_next, y_next] != plant:
                    if dx > 0:
                        x_sides.add(x_current + 1)
                    elif dx < 0:
                        x_sides.add(x_current)
                    if dy > 0:
                        y_sides.add(y_current + 1)
                    elif dy < 0:
                        y_sides.add(y_current)
            elif x_next < 0:
                print('min_x', x_current, y_current, f'dx={x_current}')
                x_sides.add(x_current)
            elif x_next >= n_rows:
                print('max_x', x_current, y_current, f'dx={x_current+1}')
                x_sides.add(x_current + 1)
            elif y_next < 0:
                print('min_y', x_current, y_current, f'dy={y_current}')
                y_sides.add(y_current)
            elif y_next >= n_cols:
                print('max_y', x_current, y_current, f'dy={y_current+1}')
                y_sides.add(y_current + 1)

    n_sides = len(x_sides) + len(y_sides)

    return n_sides

In [8]:
def compute_total_cost_p2(grid):
    total_cost = 0
    for plant, n_connexions, coords in get_all_regions(grid):
        area = len(coords)
        n_sides = cound_sides_region(grid, plant, coords)
        total_cost += area * n_sides
    return total_cost

In [9]:
def main1(file):
    with open(file) as file_in:
        grid_str = file_in.read()
    grid = np.array([list(row) for row in grid_str.splitlines()])
    total_cost = compute_total_cost_p1(grid)
    return total_cost

In [10]:
main1('input.txt')

1431440

In [11]:
grid_str = '''EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
'''

grid = np.array([list(row) for row in grid_str.splitlines()])
n_rows, n_cols = grid.shape

regions = get_all_regions(grid)
plant, n_connexions, coords = regions[0]

x_values, y_values = zip(*coords)
x_min, x_max = min(x_values) - 1, max(x_values) + 1
y_min, y_max = min(y_values) - 1, max(y_values) + 1
bounding_box = np.full((x_max - x_min + 1, y_max - y_min + 1), '.', dtype=str)
for x, y in coords:
    bounding_box[x - x_min, y - y_min] = plant

coords_dots = np.argwhere(bounding_box == '.')
start_dot = tuple(coords_dots[0].tolist())
visited_dots = set()
corners = 0

queue = deque([start_dot])
while queue:
    x_current, y_current = queue.popleft()
    if (x_current, y_current) in visited_dots:
        continue
    visited_dots.add((x_current, y_current))
    for dx, dy in directions:
        x_next, y_next = x_current + dx, y_current + dy
        if 0 <= x_next < bounding_box.shape[0] and 0 <= y_next < bounding_box.shape[1] and (x_next, y_next) not in visited_dots:
            if bounding_box[x_next, y_next] == '.':
                queue.append((x_next, y_next))

In [12]:
bounding_box

array([['.', '.', '.', '.', '.', '.', '.'],
       ['.', 'E', 'E', 'E', 'E', 'E', '.'],
       ['.', 'E', '.', '.', '.', '.', '.'],
       ['.', 'E', 'E', 'E', 'E', 'E', '.'],
       ['.', 'E', '.', '.', '.', '.', '.'],
       ['.', 'E', 'E', 'E', 'E', 'E', '.'],
       ['.', '.', '.', '.', '.', '.', '.']], dtype='<U1')